In [1]:
import sqlite3, csv

In [2]:
web_db = sqlite3.connect("web_db_3mo.db")
web_db_cur = web_db.cursor()

with open('web_db_3mo.csv') as f:
   web_db_reader = csv.reader(f)
   web_db_data = list(web_db_reader)

web_db_cur.execute('''CREATE TABLE user_clicks (
   day TEXT,
   name TEXT NOT NULL,
   clicks INTEGER
   )''')

for row in web_db_data:
    web_db_cur.execute("INSERT INTO user_clicks (day, name, clicks) values (?, ?, ?)", row)

web_db.commit()

In [3]:
crm_db = sqlite3.connect("crm_db.db")
crm_db_cur = crm_db.cursor()

with open('crm_db.csv') as f:
   crm_db_reader = csv.reader(f)
   crm_db_data = list(crm_db_reader)

crm_db_cur.execute('''CREATE TABLE user_revenue (
   user_name TEXT NOT NULL,
   start_date TEXT,   
   past_3mo_sub_revenue INTEGER
   )''')

for row in crm_db_data:
    crm_db_cur.execute("INSERT INTO user_revenue (user_name, start_date, past_3mo_sub_revenue) values (?, ?, ?)", row)

crm_db.commit()

In [4]:
web_db_cur.execute("SELECT SUM(clicks), name FROM user_clicks GROUP BY name")
user_clicks_rows = web_db_cur.fetchall()

# extract surnames
surnames = [name[1].split()[-1] for name in user_clicks_rows]

web_data_dict = dict()

for i, surname in enumerate(surnames[1:], start=1):
    web_data_dict[surname] = (user_clicks_rows[i][0], user_clicks_rows[i][1])

In [5]:
crm_db_cur.execute("SELECT * FROM user_revenue")
crm_rows = crm_db_cur.fetchall()
crm_data_dict = dict()

for surname in surnames[1:]:
    for row in crm_rows[1:]:
        if surname in row[0]:
            crm_data_dict[surname] = (row[1], row[2])

In [6]:
user_statistics_data = list()

for key in crm_data_dict:
    if key in web_data_dict:
        user_statistics_data.append(crm_data_dict[key] + web_data_dict[key])

In [7]:
user_statistics_db = sqlite3.connect("user_statistics.db")
user_statistics_db_cur = user_statistics_db.cursor()

user_statistics_db_cur.execute('''CREATE TABLE user_statistics (
   start_date TEXT,
   past_3mo_sub_revenue INTEGER,
   past_3mo_clicks INTEGER,
   name TEXT NOT NULL   
   )''')

for row in user_statistics_data:
    user_statistics_db_cur.execute("INSERT INTO user_statistics (start_date, past_3mo_sub_revenue, past_3mo_clicks, name) values (?, ?, ?, ?)", row)

user_statistics_db.commit()

In [9]:
user_statistics_db_cur.execute("SELECT * FROM user_statistics")
stat_rows = user_statistics_db_cur.fetchall()

for row in stat_rows:
    print(row)

('2000-12-01', '$3065', 98804, 'Abel Valdez')
('1998-07-09', '$3347', 98887, 'Allan Kaufman')
('2008-11-17', '$555', 105615, 'Brendan Copeland')
('2016-04-26', '$3843', 98773, 'Carlo Caldwell')
('2018-05-20', '$2329', 97492, 'Cecily Pham')
('2012-11-07', '$75', 99883, 'Christian Knight')
('2012-05-16', '$599', 97692, 'Dan Shannon')
('2010-09-17', '$673', 90904, 'Darren Zimmerman')
('2019-04-10', '$574', 99625, 'Elise Hodge')
('2005-04-01', '$57', 99094, 'Francesca Dotson')
('2009-08-14', '$2432', 97766, 'Guy Kemp')
('1991-09-04', '$405', 90210, 'Hector Jacobs')
('1990-07-16', '$2403', 94185, 'Ismail Harmon')
('1996-06-12', '$3937', 94472, 'Jared Carrillo')
('2012-04-20', '$3323', 102590, 'Leighton Austin')
('1995-04-05', '$4954', 95240, 'Leo Alexander')
('2019-04-10', '$574', 99886, 'Lorna Hodges')
('1994-05-30', '$3815', 90885, 'Lucille OConnor')
('1991-05-12', '$4768', 104235, 'Lukas Pineda')
('2013-04-01', '$115', 100532, 'Marina Nguyen')
('2002-05-14', '$4935', 101846, 'Minnie Comb